# Обучение на малом объеме данных. 7 (перенесла сюда), 10, 11 лекция

Transfer learning, Augmentation. One-shot learning. Кластеризация (embedding, one-shot learning - ПРИМЕР С ЛИЦАМИ )

Алгоритм распознавания образов обучающийся с одного раза (One-Shot learning)

https://habr.com/ru/post/414425/

### Как решить проблему меньшего количества данных?

источник: https://towardsdatascience.com/breaking-the-curse-of-small-datasets-in-machine-learning-part-1-36f28b0c044d
перевод - яндекс, могу окультурить 

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_33.png" width="700">

На приведенном выше рисунке предпринята попытка охватить основные проблемы, с которыми приходится сталкиваться при работе с небольшими наборами данных, а также возможные подходы и методы их решения. В этой части мы сосредоточимся только на методах, используемых в традиционном машинном обучении, а остальное будет обсуждаться в части 2 блога.

а) Изменение функции потерь: Для задач классификации мы часто используем кросс-энтропийные потери и редко используем среднюю абсолютную ошибку или среднеквадратичную ошибку для обучения и оптимизации нашей модели. В случае несбалансированных данных модель становится более смещенной в сторону класса большинства, поскольку она оказывает большее влияние на конечное значение потерь, и наша модель становится менее полезной. В таких сценариях мы можем добавить веса к потерям, соответствующим различным классам, чтобы выровнять это смещение данных. Например, если у нас есть два класса с данными в соотношении 4:1, мы можем применить веса в соотношении 1:4 к вычислению функции потерь, чтобы сделать данные сбалансированными. Этот метод помогает нам легко смягчить проблему несбалансированных данных и улучшает обобщение моделей в различных классах. Мы можем легко найти библиотеки как в R, так и в Python, которые могут помочь в присвоении весов классам при расчете потерь и оптимизации. Scikit-learn имеет удобную функцию полезности для вычисления весов на основе частот классов:

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', np.unique(y_train),y_train)

Мы можем избежать приведенного выше вычисления, используя class_weight=`balanced`, который выполняет те же вычисления, чтобы найти class_weights. Мы также можем подавать явные веса классов в соответствии с нашими требованиями. Для получения более подробной информации обратитесь к документации Scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create decision tree classifer object
clf = LogisticRegression(random_state=0, class_weight='balanced')

# Train model
model = clf.fit(X_std, y)

б) Обнаружение аномалий/изменений: В случаях сильно несбалансированных наборов данных, таких как мошенничество или отказ машины, стоит задуматься, можно ли рассматривать такие примеры как Аномалии или нет. Если данная задача удовлетворяет критерию аномальности, то мы можем использовать такие модели, как SVM Одного класса, методы кластеризации или методы обнаружения гауссовых аномалий. Эти методы требуют сдвига в мышлении, когда мы рассматриваем младший класс как класс выбросов, который может помочь нам найти новые способы разделения и классификации. Обнаружение изменений аналогично обнаружению аномалий, за исключением того, что мы ищем изменение или различие вместо аномалии. Это могут быть изменения в поведении пользователя, наблюдаемые с помощью шаблонов использования или банковских транзакций. Пожалуйста, обратитесь к следующей документации, чтобы узнать, как реализовать обнаружение аномалий с помощью Scikit-Learn.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_34.png" width="700">

в) Восходящая выборка или нисходящая выборка: Поскольку несбалансированные данные по своей сути наказывают класс большинства с разным весом по сравнению с классом меньшинства, одно из решений проблемы состоит в том, чтобы сделать данные сбалансированными. Это может быть сделано либо путем увеличения частоты класса меньшинства, либо путем уменьшения частоты класса большинства с помощью методов случайной или кластерной выборки. Выбор передискретизации против недискретизации и случайной против кластеризации определяется бизнес-контекстом и размером данных. Как правило, повышающая дискретизация предпочтительна, когда общий размер данных невелик, в то время как понижающая дискретизация полезна, когда у нас есть большой объем данных. Аналогично, случайная или кластеризованная выборка определяется тем, насколько хорошо распределены данные. Для более подробного понимания, пожалуйста, обратитесь к следующему блогу. Повторная выборка может быть легко выполнена с помощью пакета imdb learn, как показано ниже:

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X, y)

г) Генерировать синтетические данные: Хотя повышающая или понижающая дискретизация помогает сделать данные сбалансированными, дублирование данных увеличивает вероятность переоснащения. Другой подход к решению этой проблемы заключается в создании синтетических данных с помощью данных о меньшинствах. Метод синтетической передискретизации меньшинства (SMOTE) и Модифицированный SMOTE - это два таких метода, которые генерируют синтетические данные. Проще говоря, SMOTE берет точки данных миноритарного класса и создает новые точки данных, которые лежат между любыми двумя ближайшими точками данных, соединенными прямой линией. Для этого алгоритм вычисляет расстояние между двумя точками данных в пространстве признаков, умножает расстояние на случайное число от 0 до 1 и помещает новую точку данных на это новое расстояние от одной из точек данных, используемых для вычисления расстояния. Обратите внимание, что число ближайших соседей, рассматриваемых для генерации данных, также является гиперпараметром и может быть изменено в зависимости от требований.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_35.png" width="700">

M-SMOTE представляет собой модифицированную версию SMOTE который берет лежащее в основе распределение меньшинством во внимание. Этот алгоритм классифицирует выборки классов меньшинств на 3 различные группы — образцы безопасности/Безопасности, Пограничные выборки и скрытые шумовые выборки. Это делается путем вычисления расстояний между выборками класса меньшинства и выборками обучающих данных. В отличие от SMOTE, алгоритм случайным образом выбирает точку данных из k ближайших соседей для выборки безопасности, выбирает ближайшего соседа из граничных выборок и ничего не делает для скрытого шума. Для более подробного понимания обратитесь к блогу.

д) Методы ансамблирования: Идея объединения нескольких слабых учащихся/различных моделей показала отличные результаты при работе с несбалансированными наборами данных. Как методы упаковки в мешки, так и методы повышения эффективности показали отличные результаты в различных задачах и должны быть изучены вместе с методами, рассмотренными выше, чтобы получить лучшие результаты. В попытке ограничить длину этого блога я не буду обсуждать эти методы, но для детального понимания различных методов ансамблирования и того, как использовать их для несбалансированных данных, пожалуйста, обратитесь к следующему блогу.

### Вывод:
В этой части мы увидели, что размер данных может проявлять проблемы, связанные с обобщением, дисбалансом данных и трудностями в достижении глобального оптимума. Мы рассмотрели несколько наиболее часто используемых методов решения таких проблем для традиционных алгоритмов машинного обучения. Один или несколько из вышеперечисленных методов могут быть хорошей отправной точкой в зависимости от конкретной бизнес-задачи. Чтобы сохранить блог коротким, я не очень подробно описал многие из этих техник, но есть много замечательных ресурсов в Интернете, которые очень подробно описывают вышеперечисленные техники. В части 2 мы обсудим, как малый набор данных препятствует процессу обучения в моделях глубокого обучения и различные способы его преодоления.

## По Ганичеву (7 лекция)

1. Подготовка данных (Data Preprocessing)
2. Инициализация весов (Weight Initialization)
3. Обучение (Optimization)*
4. Пакетная нормализация (Batch Normalization)
5. Переобучение (Overfitting)*:
Регуляризация, Dropout


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_3.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_4.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_4.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_5.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_6.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_7.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_7.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_8.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_9.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_10.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_11.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_12.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_13.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_14.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_15.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_16.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_17.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_18.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_19.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_20.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_21.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_22.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_23.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_24.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_26.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_27.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_28.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_29.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_30.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_31.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_32.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_33.png" width="700">

### ДОПИНФОРМАЦИЯЯЯЯЯЯЯЯЯЯЯЯЯЯЯЯЯЯЯ

## Transfer Learning: как быстро обучить нейросеть на своих данных

источник: https://habr.com/ru/company/binarydistrict/blog/428255/

Если перед нами встает задача распознавания изображений, можно воспользоваться готовым сервисом. Однако, если нужно обучить модель на собственном наборе данных, то придется делать это самостоятельно.

Для таких типовых задач, как классификация изображений, можно воспользоваться готовой архитектурой (AlexNet, VGG, Inception, ResNet и т.д.) и обучить нейросеть на своих данных. Реализации таких сетей с помощью различных фреймворков уже существуют, так что на данном этапе можно использовать одну из них как черный ящик, не вникая глубоко в принцип её работы.

Однако, глубокие нейронные сети требовательны к большим объемам данных для сходимости обучения. И зачастую в нашей частной задаче недостаточно данных для того, чтобы хорошо натренировать все слои нейросети. Transfer Learning решает эту проблему.

### Transfer Learning для классификации изображений

Нейронные сети, которые используются для классификации, как правило, содержат N выходных нейронов в последнем слое, где N — это количество классов. Такой выходной вектор трактуется как набор вероятностей принадлежности к классу. В нашей задаче распознавания изображений еды количество классов может отличаться от того, которое было в исходном датасете. В таком случае нам придётся полностью выкинуть этот последний слой и поставить новый, с нужным количеством выходных нейронов

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_0.png" width="700">

Зачастую в конце классификационных сетей используется полносвязный слой. Так как мы заменили этот слой, использовать предобученные веса для него уже не получится. Придется тренировать его с нуля, инициализировав его веса случайными значениями. Веса для всех остальных слоев мы загружаем из предобученного снэпшота.

Существуют различные стратегии дообучения модели. Мы воспользуемся следующей: будем тренировать всю сеть из конца в конец (end-to-end), а предобученные веса не будем фиксировать, чтобы дать им немного скорректироваться и подстроиться под наши данные. Такой процесс называется тонкой настройкой (fine-tuning).

### Структурные компоненты

Для решения задачи нам понадобятся следующие компоненты:

1. Описание модели нейросети
2. Пайплайн обучения
3. Инференс пайплайн
4. Предобученные веса для этой модели
5. Данные для обучения и валидации


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_1.png" width="700">

В нашем примере компоненты (1), (2) и (3) я буду брать из собственного репозитория, который содержит максимально легковесный код — при желании с ним можно легко разобраться. Наш пример будет реализован на популярном фреймворке TensorFlow. Предобученные веса (4), подходящие под выбранный фреймворк, можно найти, если они соответствуют одной из классических архитектур. В качестве датасета (5) для демонстрации я возьму Food-101

### Модель

В качестве модели воспользуемся классической нейросетью VGG (точнее, VGG19). Несмотря на некоторые недостатки, эта модель демонстрирует довольно высокое качество. Кроме того, она легко поддается анализу. На TensorFlow Slim описание модели выглядит достаточно компактно

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim

def vgg_19(inputs,
           num_classes,
           is_training,
           scope='vgg_19',
           weight_decay=0.0005):
    with slim.arg_scope([slim.conv2d],
                activation_fn=tf.nn.relu,
                weights_regularizer=slim.l2_regularizer(weight_decay),
                biases_initializer=tf.zeros_initializer(),
                padding='SAME'):
        with tf.variable_scope(scope, 'vgg_19', [inputs]):
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = slim.max_pool2d(net, [2, 2], scope='pool1')
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2, 2], scope='pool2')
            net = slim.repeat(net, 4, slim.conv2d, 256, [3, 3], scope='conv3')
            net = slim.max_pool2d(net, [2, 2], scope='pool3')
            net = slim.repeat(net, 4, slim.conv2d, 512, [3, 3], scope='conv4')
            net = slim.max_pool2d(net, [2, 2], scope='pool4')
            net = slim.repeat(net, 4, slim.conv2d, 512, [3, 3], scope='conv5')
            net = slim.max_pool2d(net, [2, 2], scope='pool5')
            # Use conv2d instead of fully_connected layers
            net = slim.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
            net = slim.dropout(net, 0.5, is_training=is_training, scope='drop6')
            net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
            net = slim.dropout(net, 0.5, is_training=is_training, scope='drop7')
            net = slim.conv2d(net, num_classes, [1, 1], scope='fc8',
                activation_fn=None)
            net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
    return net


### Обучение модели

Код обучения модели состоит из следующих шагов:

1. Построение train/validation пайплайнов данных
2. Построение train/validation графов (сетей)
3. Надстраивание классификационной функция потерь (cross entropy loss) поверх train графа
4. Код, необходимый для вычисления точности предсказания на валидационной выборке во время обучения
5. Логика загрузки предобученных весов из снэпшота
6. Создание различных структур для обучения
7. Непосредственно сам цикл обучения (итерационная оптимизация)

Последний слой графа конструируется с нужным нам количеством нейронов и исключается из списка параметров, загружаемых из предобученного снэпшота.


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
tf.logging.set_verbosity(tf.logging.INFO)

import model
import data

###########################################################
###  Settings
###########################################################

INPUT_SIZE = 224
RANDOM_CROP_MARGIN = 10
TRAIN_EPOCHS = 20
TRAIN_BATCH_SIZE = 64
VAL_BATCH_SIZE = 128
LR_START = 0.001
LR_END = LR_START / 1e4
MOMENTUM = 0.9
VGG_PRETRAINED_CKPT = 'data/vgg_19.ckpt'
CHECKPOINT_DIR = 'checkpoints/vgg19_food'
LOG_LOSS_EVERY = 10
CALC_ACC_EVERY = 500

###########################################################
###  Build training and validation data pipelines
###########################################################

train_ds, train_iters = data.train_dataset(train_data,
    TRAIN_BATCH_SIZE, TRAIN_EPOCHS, INPUT_SIZE, RANDOM_CROP_MARGIN)
train_ds_iterator = train_ds.make_one_shot_iterator()
train_x, train_y = train_ds_iterator.get_next()

val_ds, val_iters = data.val_dataset(val_data,
    VAL_BATCH_SIZE, INPUT_SIZE)
val_ds_iterator = val_ds.make_initializable_iterator()
val_x, val_y = val_ds_iterator.get_next()

###########################################################
###  Construct training and validation graphs
###########################################################

with tf.variable_scope('', reuse=tf.AUTO_REUSE):
    train_logits = model.vgg_19(train_x, num_classes, is_training=True)
    val_logits = model.vgg_19(val_x, num_classes, is_training=False)

###########################################################
###  Construct training loss
###########################################################

loss = tf.losses.sparse_softmax_cross_entropy(
    labels=train_y, logits=train_logits)
tf.summary.scalar('loss', loss)

###########################################################
###  Construct validation accuracy
###  and related functions
###########################################################

def calc_accuracy(sess, val_logits, val_y, val_iters):
    acc_total = 0.0
    acc_denom = 0
    for i in range(val_iters):
        logits, y = sess.run((val_logits, val_y))
        y_pred = np.argmax(logits, axis=1)
        correct = np.count_nonzero(y == y_pred)
        acc_denom += y_pred.shape[0]
        acc_total += float(correct)
        tf.logging.info('Validating batch [{} / {}] correct = {}'.format(
            i, val_iters, correct))
    acc_total /= acc_denom
    return acc_total

def accuracy_summary(sess, acc_value, iteration):
    acc_summary = tf.Summary()
    acc_summary.value.add(tag="accuracy", simple_value=acc_value)
    sess._hooks[1]._summary_writer.add_summary(acc_summary, iteration)

###########################################################
###  Define set of VGG variables to restore
###  Create the Restorer
###  Define init callback (used by monitored session)
###########################################################

vars_to_restore = tf.contrib.framework.get_variables_to_restore(
    exclude=['vgg_19/fc8'])
vgg_restorer = tf.train.Saver(vars_to_restore)

def init_fn(scaffold, sess):
    vgg_restorer.restore(sess, VGG_PRETRAINED_CKPT)

###########################################################
###  Create various training structures
###########################################################

global_step = tf.train.get_or_create_global_step()
lr = tf.train.polynomial_decay(LR_START, global_step, train_iters, LR_END)
tf.summary.scalar('learning_rate', lr)
optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=MOMENTUM)
training_op = slim.learning.create_train_op(
    loss, optimizer, global_step=global_step)
scaffold = tf.train.Scaffold(init_fn=init_fn)

###########################################################
###  Create monitored session
###  Run training loop
###########################################################

with tf.train.MonitoredTrainingSession(checkpoint_dir=CHECKPOINT_DIR,
                                       save_checkpoint_secs=600,
                                       save_summaries_steps=30,
                                       scaffold=scaffold) as sess:
    start_iter = sess.run(global_step)
    for iteration in range(start_iter, train_iters):

        # Gradient Descent
        loss_value = sess.run(training_op)

        # Loss logging
        if iteration % LOG_LOSS_EVERY == 0:
            tf.logging.info('[{} / {}] Loss = {}'.format(
                iteration, train_iters, loss_value))

        # Accuracy logging
        if iteration % CALC_ACC_EVERY == 0:
            sess.run(val_ds_iterator.initializer)
            acc_value = calc_accuracy(sess, val_logits, val_y, val_iters)
            accuracy_summary(sess, acc_value, iteration)
            tf.logging.info('[{} / {}] Validation accuracy = {}'.format(
                iteration, train_iters, acc_value))

После запуска обучения можно посмотреть на его ход с помощью утилиты TensorBoard, которая поставляется в комплекте с TensorFlow и служит для визуализации различных метрик и других параметров.

In [ ]:
tensorboard --logdir checkpoints/

В конце обучения в TensorBoard мы наблюдаем практически идеальную картину: снижение Train loss и рост Validation Accuracy

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_2.png" width="700">

В результате мы получаем сохранённый снэпшот в checkpoints/vgg19_food, который будем использовать во время тестирования нашей модели (inference).

ЕСЛИ ЭТО АКТУАЛЬНО, МОЖНО ДОБАВИТЬ ЕЩЕ ИЗ ТОЙ ЖЕ СТАТЬИ

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_0.png" width="700">